In [8]:
!pip install numpy networkx

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import numpy as np
import networkx as nx

Common unit measures:
* time: seconds
* data: gigabytes

In [2]:
# NUMERICAL VALUES

# C: capacity of each link (in Gbit/s -> GB/s)
C = 10 / 8

# tau: time to move between two nodes (in microseconds -> seconds)
tau = 5 / 1000000

# L_f: size of input data (in TB -> GB)
L_f = 4 * 1000

# L_o: size of output data (in TB -> GB)
L_o = 4 * 1000

# E_X: expected value of X, the variable amount of time of the computational job (in hours -> seconds)
E_X = 8 * 3600

# T_0: fixed setup time (in seconds)
T_0 = 30

# xi: coefficient for average server time in job running cost calculation
xi = 0.1

# f: fracion of overhead of data given by TCP
f = 48 / 1500

# n: number of ports per switch
n = 64

# N_max: max number of parallel tasks
N_max = 10000

Work pipeline for no parallelism:
* Data arrives at server A
* A starts computation
* A ends computation and the response is available

$E[R] = T_0 + E[X] = 30 + 8 * 3.600 = 28.830$ seconds

$S = E[R] + \xi E[\Theta] = T_0 + E[X] + \xi (T_0 + E[X]) = (1 + \xi) * (T_0 + E[X]) = 1.1 * (28.830) = 31.713$ seconds


In [6]:
# E_R_local: mean response time for local computation (in seconds)
E_R_local = T_0 + E_X

# S_local: job running cost for local computation
S_local = int(E_R_local + xi * E_R_local)

Work pipeline with parallelism:
* Build network
* Get distances of nodes
* Data arrives at server A
* Data get sent to subservers (consider overhead)
* Get computation time of each subserver
* Get data produced to each subserver
* Get network delay time for each subserver (consider overhead)
* Get time of slowest subserver

In [40]:
def get_fat_tree_distances(N):
    same_edge_n_servers = (n // 2) - 1
    same_edge_distance = 2

    same_pod_n_servers = (n**2) // 4 - same_edge_n_servers
    same_pod_distance = 4

    inter_pod_n_servers = (n**3) // 4 - same_pod_n_servers - same_edge_n_servers
    inter_pod_distance = 6

    if(N <= same_edge_n_servers):
        return np.full(N, same_edge_distance)
    else:
        distances_array = np.full(same_edge_n_servers, same_edge_distance)

    if(N <= same_pod_n_servers + same_edge_n_servers):
        return np.concatenate((distances_array, np.full(N - same_edge_n_servers, same_pod_distance)), axis=None)
    else:
        distances_array = np.concatenate((distances_array, np.full(same_pod_n_servers, same_pod_distance)), axis=None)

    if(N <= inter_pod_n_servers + same_pod_n_servers + same_edge_n_servers):
        return np.concatenate((distances_array, np.full(N - same_edge_n_servers - same_pod_n_servers, inter_pod_distance)), axis=None)
    else:
        raise ValueError("N is too large.")
    
def get_fat_tree_rtts(N):
    distances_array = get_fat_tree_distances(N)
    rtts_array = 2 * tau * distances_array

    return rtts_array

def get_fat_tree_throughputs(N):
    rtts_array = get_fat_tree_rtts(N)
    throughputs_array = C * (1 / rtts_array) / (1 / rtts_array).sum()

    return throughputs_array

In [42]:
get_fat_tree_throughputs(100)

array([0.01908397, 0.01908397, 0.01908397, 0.01908397, 0.01908397,
       0.01908397, 0.01908397, 0.01908397, 0.01908397, 0.01908397,
       0.01908397, 0.01908397, 0.01908397, 0.01908397, 0.01908397,
       0.01908397, 0.01908397, 0.01908397, 0.01908397, 0.01908397,
       0.01908397, 0.01908397, 0.01908397, 0.01908397, 0.01908397,
       0.01908397, 0.01908397, 0.01908397, 0.01908397, 0.01908397,
       0.01908397, 0.00954198, 0.00954198, 0.00954198, 0.00954198,
       0.00954198, 0.00954198, 0.00954198, 0.00954198, 0.00954198,
       0.00954198, 0.00954198, 0.00954198, 0.00954198, 0.00954198,
       0.00954198, 0.00954198, 0.00954198, 0.00954198, 0.00954198,
       0.00954198, 0.00954198, 0.00954198, 0.00954198, 0.00954198,
       0.00954198, 0.00954198, 0.00954198, 0.00954198, 0.00954198,
       0.00954198, 0.00954198, 0.00954198, 0.00954198, 0.00954198,
       0.00954198, 0.00954198, 0.00954198, 0.00954198, 0.00954198,
       0.00954198, 0.00954198, 0.00954198, 0.00954198, 0.00954